<a href="https://colab.research.google.com/github/jespimentel/analisador_lote_ip_colab/blob/main/analisador_custodia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ANALISADOR DE APFD EM LOTE PARA AUDIÊNCIAS DE CUSTÓDIA
### José Eduardo de Souza Pimentel
---
### Visão geral
O programa emprega o poder do Python e das LLMs, mediadas pela biblioteca `litellm`, para analisar uma coleção de autos de prisão em flagrante em formato PDF encontrados em determinada pasta (diretório).

Fique à vontade para alterar o programa e adaptá-lo às suas necessidades.

### Estratégias
1. Usamos o `PyPDF` para extrair textos dos PDFs. Preservamos a numeração das páginas para referenciar as respostas.
2. O texto extraído compõe um extenso prompt, com as perguntas à LLM dirigidas à analise de cada caso.
3. A biblioteca `litellm` facilita o desenvolvimento e abstrai toda a complexidade das chamadas a APIs, tornando fácil substituir um provedor de LLM por outro.
4. Por fim, usamos a biblioteca `Spire.Doc` para converter o relatório gerado pela LLM em um documento `docx` na pasta de trabalho.

---

# Como usar?

1. Crie a chave de API no seu provedor de IA de preferência.
2. Conecte este Colab ao seu GoogleDrive.
3. Copie o caminho para a pasta de trabalho, onde serão carregados os PDFs.
4. Edite o código para incluir a chave de API e o caminho para a pasta.
5. Faça as alterações no prompt para adaptá-lo às suas necessidades.
6. Rode o `Executar tudo`. No menu, aparece sob `Ambiente de execução`.
7. Confira o resultado no documento gerado na pasta de trabalho.

Obs. Qualquer dúvida, procure pelo tutorial no meu canal do YouTube: https://www.youtube.com/@jespimentel


# Instalações e importações necessárias

In [1]:
# Instalações
!pip install PyPDF2 litellm
!pip install Spire.Doc

In [2]:
# Importações

import os
import PyPDF2
import re

from google.colab import drive, userdata
#drive.mount('/content/drive')

from litellm import completion

# Configurações do usuário

In [3]:
# Configurações gerais

modelo = 'gpt-4o-mini'
api_key = userdata.get('OPENAI_KEY') # Se você não for compartilhar o código, use: api_key = 'sk-xyz...lko'

#modelo = 'gemini/gemini-1.5-flash'
#api_key = userdata.get('GOOGLE_KEY')

#modelo = 'deepseek/deepseek-chat'
#api_key = userdata.get('DEEPSEEK_KEY')

# Teste com outros modelos de sua preferência
#modelo = ''
#api_key = ''

pasta_de_trabalho = "/content/drive/MyDrive/promotoria/pdfs" # Ajuste conforme sua necessidade

In [4]:
# Configurações de prompt

prompt = f"""
O texto fornecido corresponde a um auto de prisão em flagrante.

Você vai analisar e fornecer as seguintes informações:

* Número do flagrante (use o padrão CNJ, exemplo: 1502524-61.2024.8.26.0599):

* Nome do indiciado

* Breve resumo da ocorrência (com indicação de data, horário, local e demais informações, de até 2 parágrafos em texto corrido)

* Prova material

* Informar se o indiciado reclamou do tratamento dispensado pelos policiais durante a prisão

* Informar se o indiciado sofreu lesão corporal

* Informar se o indiciado possui antecedentes criminais


Instruções especiais:

* Apresente as informações de forma estruturada.

* Utilize linguagem jurídica formal e precisa.

* Não emita opiniões pessoais ou julgamentos de valor.

* Em caso de mais de um indiciado, certifique-se de que abordou as questões acima para cada um deles

"""

# Programa

In [5]:
# Funções

def listar_pdfs(diretorio):
    pdfs = []
    for arquivo in os.listdir(diretorio):
        if arquivo.lower().endswith(".pdf"):
            pdfs.append(os.path.join(diretorio, arquivo))
    return pdfs

def ler_pdf(caminho_pdf):
    with open(caminho_pdf, 'rb') as arquivo:
        leitor = PyPDF2.PdfReader(arquivo)
        texto = ''
        for pagina_num, pagina in enumerate(leitor.pages, start=1):
            texto += f"--- Página {pagina_num} ---\n"
            texto += pagina.extract_text() + "\n\n"
    return texto

def limpar_texto(texto):
  # Remover a assinatura
  padrao_assinatura = r"Este documento .*?fls\. \d+"
  texto = re.sub(padrao_assinatura, "", texto, flags=re.DOTALL)
  # Remover sequências de números e barras
  padrao_sequencias = r"\/[j\d]+ (?:\/[j\d]+)+"
  texto_limpo = re.sub(padrao_sequencias, "", texto)
  return texto_limpo

def analisar_conteudo (texto, prompt, modelo, api_key):
  try:
    resultado = completion(
    model=modelo,
    messages=[{"role": "system", "content": "Você é um analista jurídico especializado em Direito Penal e Processo Penal."},
                {"role": "system", "content": "Analise o inquérito policial com base somente no texto fornecido."},
                {"role": "user", "content": f"**Texto extraído do inquérito policial:**\n\n{texto}"},
                {"role": "user", "content": f"**Instruções:**\n {prompt}"}],
                api_key=api_key,
                temperature=0.2,
                #max_tokens=64000,
    )
    return resultado.get('choices', [{}])[0].get('message', {}).get('content', 'Sem resposta.')
  except Exception as e:
    print(f"Erro: {e}")
    return None

def gerar_markdown(texto, nome_arquivo="relatorio.md"):
  modo = 'a' if os.path.exists(nome_arquivo) else 'w'
  try:
    with open(nome_arquivo, modo, encoding='utf-8') as arquivo:
      if modo == 'w':
        arquivo.write("# PROMOTORIA DE JUSTIÇA DE PIRACICABA\n\n")
        arquivo.write(texto)
      else:
         arquivo.write(texto)
  except Exception as e:
    print(f"Erro ao salvar o arquivo: {e}")

In [ ]:
pdfs = listar_pdfs(pasta_de_trabalho)
for pdf in pdfs:
    texto = ler_pdf(pdf)
    texto_limpo = limpar_texto(texto)
    resultado = analisar_conteudo(texto_limpo, prompt, modelo, api_key)
    if resultado:
        conteudo = f"\n**Arquivo:** {pdf}\n\n"
        conteudo += resultado
        conteudo += "\n___________________________________\n"
        print(conteudo)
        gerar_markdown(conteudo)
    else:
        print(f"Não foi possível obter o resultado de {pdf}.")

# Salvando o relatório

In [7]:
from spire.doc import *
from spire.doc.common import *

document = Document()
document.LoadFromFile("relatorio.md")
document.SaveToFile(f"{pasta_de_trabalho}/relatorio.docx", FileFormat.Docx2016) # Altere para .pdf e FileFormat.PDF, se preferir
document.Dispose()